# Lab 11: Part 2 - Food Inspection Forecasting: Data processing
This file is an ipython notebook with [`R-magic`](https://ipython.org/ipython-doc/1/config/extensions/rmagic.html) to convert the data from Rds (the R programming language data storage sytem) to `csv` to be read into Python. If you ever find yourself in a bind with R code available for you... give `R-magic` a try. 


## **HUGE NOTE:  All code here is taken from the [food-inspections-evaluation]( https://github.com/Chicago/food-inspections-evaluation) repository** 
### They did a great job at cleaning the data in R so I don't want to repeat work.

All code and data is available on GitHub:
https://github.com/Chicago/food-inspections-evaluation

#### How to get R running in a Python Jupyter Notebook (Windows10)?
**January 2021**. To get `Lab11-Project-and-Food-Inspection-Forecasting.ipynb` up and running, it was inevitable to perform this data processing part of Lab11: I couldn't circumvent it because the links to the CSV files `model_matrix.csv`, `target.csv` (outcome) and `food_inspections.csv` are not valid anymore and searching for these files on [CS109](http://cs109.github.io/2015/) and the internet was unsuccessful.<br>See below the efforts done to make this notebook working:
- added a cell (the 1st one below) to install `rpy2` using `pip`. Gave errors executing `%load_ext rpy2.ipython`.
- in the Anaconda prompt terminal installed `rpy2` using `conda install conda install -c r rpy2`. Installation failed but succeeded with `conda install -c conda-forge rpy2`. Gave errors executing `%load_ext rpy2.ipython` indicating entry points to procedures in DLL files could not be found. Well, all DLL files are available and accessible.
- extended PATH variable with path to these DLL files. After Kernel restart same errors as before.
- based on information described as a solution three additional environment vars added: R_HOME, R_LIBS_USER and R_USER. After Kernel restart still same errors as before.
- after reading [set up R and Python in same Jupyter Notebook](https://anaconda.zendesk.com/hc/en-us/articles/360023857134-Setting-up-rpy2-on-Windows) the PATH variable was extended with some Anaconda3 paths. After Kernel restart still same errors as before.
- thank heavens a computer restart apparently kicked some process to make `%load_ext rpy2.ipython` finally work. So actually unknown which of the above trial-and-error solutions did the trick.

In [ ]:
pip install rpy2

In [1]:
import rpy2
import pandas as pd
%load_ext rpy2.ipython

In [7]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [39]:
%%R
# change to your local clone
#data_dir = '~/food-inspections-evaluation/'
#out_dir = '~'

data_dir <- "C:/DATA/henk/Werk/DATA SCIENCE/Harvard CS109/CS109-2015/Labs/2015lab11/data/food-inspections-evaluation"
out_dir <- "C:/DATA/henk/Werk/DATA SCIENCE/Harvard CS109/CS109-2015/Labs/2015lab11/data/out"

library("data.table", "ggplot2")

setwd(data_dir)

Just to be sure the absolute paths have been given. Package `data.table` yet had to be loaded: installation from **RStudio**: `install.packages("data.table", dependencies=TRUE)`. <br> The required Rds files were downloaded from the [Chicago repository](https://github.com/Chicago/food-inspections-evaluation) to the `food-inspections-evaluation` folder. I checked out from a 2015 version.

### Food Inspection database processing

In [35]:
%%R
food <- readRDS('food_inspections.Rds')
write.csv(food, file = paste(out_dir, '/food_inspections.csv', sep = ''), row.names = FALSE)

### Model Dataframe processing

In [36]:
%%R
dat = readRDS('dat_model.Rds')
write.csv(dat, file = paste(out_dir, '/dat_model.csv', sep = ''))

In [46]:
%%R
dat <- readRDS("dat_model.Rds")

## Only keep "Retail Food Establishment"
dat <- dat[LICENSE_DESCRIPTION == "Retail Food Establishment"]
## Remove License Description
dat$LICENSE_DESCRIPTION <- NULL
dat <- na.omit(dat)

## Add criticalFound variable to dat:
dat$criticalFound <- pmin(1, dat$criticalCount)

# ## Set the key for dat
setkey(dat, Inspection_ID)

# Match time period of original results
# dat <- dat[Inspection_Date < "2013-09-01" | Inspection_Date > "2014-07-01"]

#==============================================================================
# CREATE MODEL DATA
#==============================================================================
# sort(colnames(dat))

xmat <- dat[ , list(Inspector = Inspector_Assigned,
                    pastSerious = pmin(pastSerious, 1),
                    pastCritical = pmin(pastCritical, 1),
                    timeSinceLast,
                    ageAtInspection = ifelse(ageAtInspection > 4, 1L, 0L),
                    consumption_on_premises_incidental_activity,
                    tobacco_retail_over_counter,
                    temperatureMax,
                    heat_burglary = pmin(heat_burglary, 70),
                    heat_sanitation = pmin(heat_sanitation, 70),
                    heat_garbage = pmin(heat_garbage, 50),
                    # Facility_Type,
                    criticalFound),
            keyby = Inspection_ID]
mm <- model.matrix(criticalFound ~ . -1, data=xmat[ , -1, with=F])
mm <- as.data.table(mm)
str(mm)
colnames(mm)

#==============================================================================
# CREATE TEST / TRAIN PARTITIONS
#==============================================================================
# 2014-07-01 is an easy separator

dat[Inspection_Date < "2014-07-01", range(Inspection_Date)]
dat[Inspection_Date > "2014-07-01", range(Inspection_Date)]

iiTrain <- dat[ , which(Inspection_Date < "2014-07-01")]
iiTest <- dat[ , which(Inspection_Date > "2014-07-01")]

## Check to see if any rows didn't make it through the model.matrix formula
nrow(dat)
nrow(xmat)
nrow(mm)


Classes 'data.table' and 'data.frame':	18712 obs. of  16 variables:
 $ Inspectorblue                              : num  0 1 1 1 1 0 0 0 0 0 ...
 $ Inspectorbrown                             : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Inspectorgreen                             : num  1 0 0 0 0 0 0 0 0 0 ...
 $ Inspectororange                            : num  0 0 0 0 0 1 1 1 1 1 ...
 $ Inspectorpurple                            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ Inspectoryellow                            : num  0 0 0 0 0 0 0 0 0 0 ...
 $ pastSerious                                : num  0 0 0 0 0 0 0 0 0 0 ...
 $ pastCritical                               : num  0 0 0 0 0 0 0 0 0 0 ...
 $ timeSinceLast                              : num  2 2 2 2 2 2 2 2 2 2 ...
 $ ageAtInspection                            : num  1 1 1 1 1 1 0 1 1 0 ...
 $ consumption_on_premises_incidental_activity: num  0 0 0 0 0 0 0 0 0 0 ...
 $ tobacco_retail_over_counter                : num  1 0 0 0 0 0 0 0 0 0 ...
 $ tempe

In [47]:
%%R
# Output Model Matrix and Target
write.csv(mm, file = paste(out_dir, '/model_matrix.csv', sep = ''), row.names = FALSE)
write.csv(xmat$criticalFound, file = paste(out_dir, '/TARGET.csv', sep = ''), row.names = FALSE)